In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve
#-----
from sklearn import svm
from sklearn.linear_model import LogisticRegression
#-----
import scipy
from scipy import signal
#-----
from tensorflow import keras
#-----
# from tslearn.shapelets import LearningShapelets

In [3]:
from itertools import product
def get_outg_confusion_matrix_text_colors(
    cmd
):
    r"""
    Basically taken from sklearn.metrics.ConfusionMatrixDisplay.plot
    Colors are needed so my additional text matches what's provided by sklearn method.
    
    SHOULD BE CALLED AFTER sklearn.metrics.ConfusionMatrixDisplay.plot!
    """
    #-------------------------
    cm = cmd.confusion_matrix
    assert(cm.shape[0]==2)
    n_classes = cm.shape[0]
    #-------------------------
    cmap_min, cmap_max = cmd.im_.cmap(0), cmd.im_.cmap(1.0)
    thresh = (cm.max() + cm.min()) / 2.0
    #-------------------------
    colors = np.empty((2,2), dtype=object)
    for i,j in product(range(2), range(2)):
        color = cmap_max if cm[i, j] < thresh else cmap_min
        colors[i,j] = color
    #-------------------------
    return colors

In [4]:
def draw_confusion_matrix(
    y, 
    y_pred, 
    title=None, 
    normalize=None, 
    scientific=True, 
    ax=None, 
    text_kw=dict(fontsize='xx-large'), 
    target_eq_1_name='Outage', 
    target_eq_0_name='Baseline'
):
    r"""
    Visualize confusion matrix for outages using sklearn.metrics.ConfusionMatrixDisplay
    """
    #-------------------------
    cmd = ConfusionMatrixDisplay(
        confusion_matrix(y, y_pred, normalize=normalize), 
        display_labels=[target_eq_0_name, target_eq_1_name]
    )
    #-----
    if scientific:
        cmd.plot(values_format='.3e', ax=ax, text_kw=text_kw)
    else:
        cmd.plot(values_format='', ax=ax, text_kw=text_kw)
    #-------------------------
    cmd.ax_.set_xlabel('Predicted', fontsize='xx-large')
    cmd.ax_.set_ylabel('True', fontsize='xx-large')
    cmd.ax_.set_title(title, fontsize='xx-large', fontweight='semibold')
    #ax.set_size_inches(12, 10)
    #-------------------------
    if scientific:
        txt_fmt     = '{:.4e}'
        pct_txt_fmt = txt_fmt
    else:
        txt_fmt     = '{}'
        pct_txt_fmt = '{:.4f}'
    #-----
    ax = cmd.ax_
    ax.text(1.5, 0.9, "# Entries:  {}".format(txt_fmt).format(y.shape[0]), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.8, "# {}:  {}".format(target_eq_1_name, txt_fmt).format(y.sum()), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.7, "# {}: {}".format(target_eq_0_name, txt_fmt).format(y.shape[0]-y.sum()), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.6, "% {}:  {}".format(target_eq_1_name, pct_txt_fmt).format(100*y.sum()/y.shape[0]), fontsize=14, transform=ax.transAxes)
    #-------------------------
    ax.text(1.5, 0.4, "ACCURACY:  {:.4f}".format(accuracy_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.3, "PRECISION:  {:.4f}".format(precision_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.2, "RECALL:       {:.4f}".format(recall_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.1, "F1:               {:.4f}".format(f1_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    #--------------------------------------------------
    # Include TN, FP, FN, TP labels
    # NOTES:
    #   Text stored in cmd.text_ is stored in row-major fashion
    #   Therefore, when plotting the value, the y-value corresponds to the 0th
    #     index and the x-value corresponds to the 1st index
    #
    #   Axes are defined with limits:
    #     x_lim = (-0.5, 1.5)
    #     y_lim = (1.5, -0.5)
    #   The axes are defined such that: 
    #     top-left corner     = (-0.5, -0.5)
    #     bottom-right corner = (1.5, 1.5) 
    #-----
    colors = get_outg_confusion_matrix_text_colors(cmd)
    cat_fontsize = text_kw.get('fontsize', 'xx-large')
    #-----
    cmd.ax_.text(0,   -0.25, 'TN', ha='center', va='center', color=colors[0,0], fontweight='bold', fontsize=cat_fontsize)
    cmd.ax_.text(1.0, -0.25, 'FP', ha='center', va='center', color=colors[0,1], fontweight='bold', fontsize=cat_fontsize)

    cmd.ax_.text(0,    0.75, 'FN', ha='center', va='center', color=colors[1,0], fontweight='bold', fontsize=cat_fontsize)
    cmd.ax_.text(1.0,  0.75, 'TP', ha='center', va='center', color=colors[1,1], fontweight='bold', fontsize=cat_fontsize)    
    #--------------------------------------------------
    #return ax, cmd.ax_
    return cmd

In [5]:
def remove_ev_submeter_in_pair(
    ami_df_i, 
    pct_0_thresh_main=0.1, 
    pct_0_thresh_subm=0.6, 
    enforce_corr=True, 
    corr_thresh=0.5, 
    #remove_undetermined=True, 
    value_col='value', 
    time_col='index', 
    PN_col='aep_premise_nb', 
    SN_col='serialnumber'
):
    r"""
    Given a pair of meter connected to a single premise, this tries to determine which is the EV submeter in the pair, 
      and keeps that which is not.
    Basically, the submeter only monitors the EV, whereas the other monitors the usage of the entire premise.
    Thus, the EV signal should be 0 for a majority of the time, and the two meters should be correlated.
    
    This is a somewhat specialized function for use with engineering the EV dataset
    
    ami_df_i:
        An AMI DF containing data for a single premise which has two meters
        
    pct_0_thresh_main:
        The maximum allowed percentage of 0 values for a meter to be considered the main meter
    pct_0_thresh_subm:
        The minimum allowed percentage of 0 values for a meter to be considered the submeter
    """
    #-------------------------
    assert(ami_df_i[PN_col].nunique()==1)
    assert(pct_0_thresh_main < pct_0_thresh_subm)
    #-------------------------
    SNs_i = ami_df_i[SN_col].unique().tolist()
    assert(len(SNs_i)<=2)
    #-------------------------
    if len(SNs_i)==1:
        return ami_df_i
    #-------------------------
    ami_df_i_1 = ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    ami_df_i_2 = ami_df_i[ami_df_i[SN_col]==SNs_i[1]]
    if time_col=='index':
        ami_df_i_1 = ami_df_i_1.sort_index()
        ami_df_i_2 = ami_df_i_2.sort_index()
    else:
        ami_df_i_1 = ami_df_i_1.sort_values(by=[time_col])
        ami_df_i_2 = ami_df_i_2.sort_values(by=[time_col])
    #--------------------------------------------------
    # NOTE: I have seen some SNs which have a bunch of 0.002 values, which, for the purposes here should be treated as 0s
    #       This is why I use <0.005 instead of ==0 and >=0.005 instead of !=0 below
    #--------------------------------------------------
    pct_1 = (ami_df_i_1[value_col]<0.005).sum()/ami_df_i_1.shape[0]
    pct_2 = (ami_df_i_2[value_col]<0.005).sum()/ami_df_i_2.shape[0]
    #-------------------------
    if pct_1 <= pct_0_thresh_main:
        defn_1 = 1
    elif pct_1 >= pct_0_thresh_subm:
        defn_1 = 0
    else:
        defn_1 = -1
    #-----
    if pct_2 <= pct_0_thresh_main:
        defn_2 = 1
    elif pct_2 >= pct_0_thresh_subm:
        defn_2 = 0
    else:
        defn_2 = -1
    #-------------------------
    if defn_1+defn_2==1:
        pass_pcts=True
    else:
        pass_pcts=False
    #-------------------------
    if not pass_pcts:
        return
    
    #--------------------------------------------------
    pass_corr=True
    if enforce_corr:
        if time_col=='index':
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_index=True, right_index=True, how='outer')
        else:
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_on=time_col, right_on=time_col, how='outer')
        # For the correlation only using values which are non-zero (and aligning, meaning non-NaN)
        ami_df_i_12 = ami_df_i_12[(ami_df_i_12[f'{value_col}_x']>=0.005) & (ami_df_i_12[f'{value_col}_y']>=0.005)]
        corr_val = ami_df_i_12[f'{value_col}_x'].corr(ami_df_i_12[f'{value_col}_y'])
        if corr_val >= corr_thresh:
            pass_corr = True
        else:
            pass_corr = False
            
    #--------------------------------------------------
    if not (pass_pcts and pass_corr):
        return
    #-------------------------
    if defn_1==1:
        assert(defn_2==0) # Sanity check, not needed
        return ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    else:
        assert(defn_1==0 and defn_2==1) # Sanity check, not needed
        return ami_df_i[ami_df_i[SN_col]==SNs_i[1]]

In [6]:
def select_pairs_w_submeter(
    ami_df_i, 
    pct_0_thresh_main=0.1, 
    pct_0_thresh_subm=0.6, 
    enforce_corr=True, 
    corr_thresh=0.5, 
    #remove_undetermined=True, 
    value_col='value', 
    time_col='index', 
    PN_col='aep_premise_nb', 
    SN_col='serialnumber'
):
    r"""
    Given a pair of meter connected to a single premise, this tries to determine which is the EV submeter in the pair, 
      and keeps that which is not.
    Basically, the submeter only monitors the EV, whereas the other monitors the usage of the entire premise.
    Thus, the EV signal should be 0 for a majority of the time, and the two meters should be correlated.
    
    This is a somewhat specialized function for use with engineering the EV dataset
    
    ami_df_i:
        An AMI DF containing data for a single premise which has two meters
        
    pct_0_thresh_main:
        The maximum allowed percentage of 0 values for a meter to be considered the main meter
    pct_0_thresh_subm:
        The minimum allowed percentage of 0 values for a meter to be considered the submeter
    """
    #-------------------------
    assert(ami_df_i[PN_col].nunique()==1)
    assert(pct_0_thresh_main < pct_0_thresh_subm)
    #-------------------------
    SNs_i = ami_df_i[SN_col].unique().tolist()
    assert(len(SNs_i)<=2)
    #-------------------------
    if len(SNs_i)==1:
        return 
    #-------------------------
    ami_df_i_1 = ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    ami_df_i_2 = ami_df_i[ami_df_i[SN_col]==SNs_i[1]]
    if time_col=='index':
        ami_df_i_1 = ami_df_i_1.sort_index()
        ami_df_i_2 = ami_df_i_2.sort_index()
    else:
        ami_df_i_1 = ami_df_i_1.sort_values(by=[time_col])
        ami_df_i_2 = ami_df_i_2.sort_values(by=[time_col])
    #--------------------------------------------------
    # NOTE: I have seen some SNs which have a bunch of 0.002 values, which, for the purposes here should be treated as 0s
    #       This is why I use <0.005 instead of ==0 and >=0.005 instead of !=0 below
    #--------------------------------------------------
    pct_1 = (ami_df_i_1[value_col]<0.005).sum()/ami_df_i_1.shape[0]
    pct_2 = (ami_df_i_2[value_col]<0.005).sum()/ami_df_i_2.shape[0]
    #-------------------------
    if pct_1 <= pct_0_thresh_main:
        defn_1 = 1
    elif pct_1 >= pct_0_thresh_subm:
        defn_1 = 0
    else:
        defn_1 = -1
    #-----
    if pct_2 <= pct_0_thresh_main:
        defn_2 = 1
    elif pct_2 >= pct_0_thresh_subm:
        defn_2 = 0
    else:
        defn_2 = -1
    #-------------------------
    if defn_1+defn_2==1:
        pass_pcts=True
    else:
        pass_pcts=False
    #-------------------------
    if not pass_pcts:
        return
    
    #--------------------------------------------------
    pass_corr=True
    if enforce_corr:
        if time_col=='index':
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_index=True, right_index=True, how='outer')
        else:
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_on=time_col, right_on=time_col, how='outer')
        # For the correlation only using values which are non-zero (and aligning, meaning non-NaN)
        ami_df_i_12 = ami_df_i_12[(ami_df_i_12[f'{value_col}_x']>=0.005) & (ami_df_i_12[f'{value_col}_y']>=0.005)]
        corr_val = ami_df_i_12[f'{value_col}_x'].corr(ami_df_i_12[f'{value_col}_y'])
        if corr_val >= corr_thresh:
            pass_corr = True
        else:
            pass_corr = False
            
    #--------------------------------------------------
    if not (pass_pcts and pass_corr):
        return
    #-------------------------
    if defn_1==1:
        assert(defn_2==0) # Sanity check, not needed
        ami_df_i_main = ami_df_i[ami_df_i[SN_col]==SNs_i[0]].copy()
        ami_df_i_subm = ami_df_i[ami_df_i[SN_col]==SNs_i[1]].copy()
    else:
        assert(defn_1==0 and defn_2==1) # Sanity check, not needed
        ami_df_i_main =  ami_df_i[ami_df_i[SN_col]==SNs_i[1]].copy()
        ami_df_i_subm =  ami_df_i[ami_df_i[SN_col]==SNs_i[0]].copy()
    #-------------------------
    if time_col=='index':
        return_ami_df_i = pd.merge(
            ami_df_i_main, 
            ami_df_i_subm[[value_col, SN_col]], 
            left_index=True, 
            right_index=True, 
            how='inner'
        )
    else:
        return_ami_df_i = pd.merge(
            ami_df_i_main, 
            ami_df_i_subm[[value_col, time_col, SN_col]], 
            left_on=time_col, 
            right_on=time_col, 
            how='inner'
        )
    #----------
    return_ami_df_i = return_ami_df_i.rename(columns={
        f'{value_col}_x': f'{value_col}_main', 
        f'{value_col}_y': f'{value_col}_subm', 
        f'{SN_col}_x': f'{SN_col}_main', 
        f'{SN_col}_y': f'{SN_col}_subm', 
    })
    #----------
    return_ami_df_i[f'{value_col}_delt'] = return_ami_df_i[f'{value_col}_main']-return_ami_df_i[f'{value_col}_subm']
    #-------------------------
    return return_ami_df_i

# SEE WEBPAGES
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/43512887#43512887
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/56451135#56451135

In [7]:
# class real_time_peak_detection():
#     def __init__(self, array, lag, threshold, influence):
#         self.y = list(array)
#         self.length = len(self.y)
#         self.lag = lag
#         self.threshold = threshold
#         self.influence = influence
#         self.signals = [0] * len(self.y)
#         self.filteredY = np.array(self.y).tolist()
#         self.avgFilter = [0] * len(self.y)
#         self.stdFilter = [0] * len(self.y)
#         self.avgFilter[self.lag - 1] = np.mean(self.y[0:self.lag]).tolist()
#         self.stdFilter[self.lag - 1] = np.std(self.y[0:self.lag]).tolist()

#     def thresholding_algo(self, new_value):
#         self.y.append(new_value)
#         i = len(self.y) - 1
#         self.length = len(self.y)
#         if i < self.lag:
#             return 0
#         elif i == self.lag:
#             self.signals = [0] * len(self.y)
#             self.filteredY = np.array(self.y).tolist()
#             self.avgFilter = [0] * len(self.y)
#             self.stdFilter = [0] * len(self.y)
#             self.avgFilter[self.lag] = np.mean(self.y[0:self.lag]).tolist()
#             self.stdFilter[self.lag] = np.std(self.y[0:self.lag]).tolist()
#             return 0

#         self.signals += [0]
#         self.filteredY += [0]
#         self.avgFilter += [0]
#         self.stdFilter += [0]

#         if abs(self.y[i] - self.avgFilter[i - 1]) > (self.threshold * self.stdFilter[i - 1]):

#             if self.y[i] > self.avgFilter[i - 1]:
#                 self.signals[i] = 1
#             else:
#                 self.signals[i] = -1

#             self.filteredY[i] = self.influence * self.y[i] + \
#                 (1 - self.influence) * self.filteredY[i - 1]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])
#         else:
#             self.signals[i] = 0
#             self.filteredY[i] = self.y[i]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])

#         return self.signals[i]


def thresholding_algo_OLD(y, lag, threshold, influence):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))


def thresholding_algo(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [8]:
def thresholding_algo_median(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.median(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.median(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.median(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [9]:
def mad(y):
    return np.mean(np.abs(y - np.mean(y)))

def thresholding_algo_mad(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(mad(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = mad(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = mad(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))


def thresholding_algo_median_mad(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.median(y)),
                    stdFilter = np.asarray(mad(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.median(y[0:lag])
    stdFilter[lag - 1] = mad(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.median(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
        stdFilter[i] = mad(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [10]:
def set_signal_groups_in_df_i(
    df_i, 
    SN_col='serialnumber',
    signal_col='signals', 
    return_signal_group_col='signal_grp'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)    
    #-------------------------    
    drop_idx = False
    if df_i.index.name in df_i.columns:
        drop_idx = True
    signals_df_i = df_i.reset_index(drop=drop_idx)[df_i.reset_index(drop=drop_idx)[signal_col]==1]
    #-----
    tmp_signal_grp_col = Utilities.generate_random_string()
    signals_df_i[tmp_signal_grp_col] = np.nan
    #-----
    tmp_idx_col = Utilities.generate_random_string()
    signals_df_i[tmp_idx_col] = signals_df_i.index
    #-----
    signals_df_i[tmp_signal_grp_col] = signals_df_i[tmp_idx_col].diff().ne(1).cumsum()
    #-------------------------
    return_df = df_i.copy()
    return_df[return_signal_group_col] = np.nan
    return_df.iloc[signals_df_i.index,-1] = signals_df_i[tmp_signal_grp_col]
    #-------------------------
    return return_df

def find_signals_and_set_signal_groups_in_df_i(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber',
    signal_col='signals', 
    return_signal_group_col='signal_grp'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    rtpd_i = thresholding_algo(
        y=df_i[value_col].tolist(), 
        lag=lag,
        threshold=threshold, 
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold
    )
    df_i[signal_col] = rtpd_i['signals']    
    #-------------------------
    return_df = set_signal_groups_in_df_i(
        df_i=df_i, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=return_signal_group_col
    )
    #-------------------------
    return return_df

In [11]:
def find_signals_and_build_df_i_signals_gpd(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    signal_group_col='signal_grp'
    df_i = find_signals_and_set_signal_groups_in_df_i(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=signal_group_col
    )
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()]
    df_i_signals = df_i_signals.drop(columns=[signal_col])
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    df_i_signals = df_i_signals.reset_index()
    assert(time_col in df_i_signals.columns.tolist())
    #-------------------------
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    return df_i_signals_gpd

In [12]:
def extract_features_for_SN(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min
    
    df_i_signals_gpd = find_signals_and_build_df_i_signals_gpd(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col, 
        time_col=time_col, 
        signal_col=signal_col
    )
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_max_mean':     peak_max_mean, 
        'peak_width_mean':   peak_width_mean, 
        'peak_spacing_mean': peak_spacing_mean, 
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [13]:
def extract_features_for_SN_v2(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    signal_group_col='signal_grp'
    df_i = find_signals_and_set_signal_groups_in_df_i(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=signal_group_col
    )
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()]
    df_i_signals = df_i_signals.drop(columns=[signal_col])
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    #-----   
    drop_idx = False
    if df_i_signals.index.name in df_i_signals.columns:
        drop_idx = True
    df_i_signals = df_i_signals.reset_index(drop=drop_idx)
    assert(time_col in df_i_signals.columns.tolist())
    #-------------------------
    # Features split into:
    #   1. Using all data in peaks pooled together
    #   2. First grouping by signal group, then extracting features
    #-------------------------
    # Features (1):
    peak_mean = df_i_signals[value_col].mean()
    peak_std  = df_i_signals[value_col].std()
    #-------------------------
    # Features (2):
    #-----
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_max_std      = df_i_signals_gpd[(value_col, 'max')].std()
    #-----
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_width_std    = df_i_signals_gpd[(time_col, 'max_m_min')].std()
    #-----
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    peak_spacing_std  = df_i_signals_gpd[(time_col, 'mean')].diff().std()
    #-----
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_mean':         peak_mean, 
        'peak_std':          peak_std, 
        #-----
        'peak_max_mean':     peak_max_mean, 
        'peak_max_std':      peak_max_std, 
        #-----
        'peak_width_mean':   peak_width_mean, 
        'peak_width_std':    peak_width_std, 
        #-----
        'peak_spacing_mean': peak_spacing_mean,
        'peak_spacing_std':  peak_spacing_std,
        #-----
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [14]:
def fill_na_in_peak_df(
    peak_df
):
    r"""
    """
    #-------------------------
    if 'peak_mean' in peak_df.columns:
        peak_df['peak_mean']         = peak_df['peak_mean'].fillna(0)
    #-----
    if 'peak_std' in peak_df.columns:
        peak_df['peak_std']          = peak_df['peak_std'].fillna(-1)
    #-------------------------
    if 'peak_max_mean' in peak_df.columns:
        peak_df['peak_max_mean']     = peak_df['peak_max_mean'].fillna(0)
    #-----
    if 'peak_max_std' in peak_df.columns:
        peak_df['peak_max_std']      = peak_df['peak_max_std'].fillna(-1)
    #-------------------------    
    if 'peak_width_mean' in peak_df.columns:
        peak_df['peak_width_mean']   = peak_df['peak_width_mean'].fillna(pd.Timedelta(0))
#         peak_df['peak_width_mean']   = peak_df['peak_width_mean'].fillna(pd.Timedelta.max)
    #-----
    if 'peak_width_std' in peak_df.columns:
        peak_df['peak_width_std']    = peak_df['peak_width_std'].fillna(pd.Timedelta(-1))
#         peak_df['peak_width_std']    = peak_df['peak_width_std'].fillna(pd.Timedelta(0))
#         peak_df['peak_width_std']    = peak_df['peak_width_std'].fillna(pd.Timedelta(pd.Timedelta.max))
    #-------------------------
    if 'peak_spacing_mean' in peak_df.columns:
        peak_df['peak_spacing_mean'] = peak_df['peak_spacing_mean'].fillna(pd.Timedelta(0))
#         peak_df['peak_spacing_mean'] = peak_df['peak_spacing_mean'].fillna(pd.Timedelta.max)
    #-----
    if 'peak_spacing_std' in peak_df.columns:
        peak_df['peak_spacing_std']  = peak_df['peak_spacing_std'].fillna(pd.Timedelta(-1))
#         peak_df['peak_spacing_std']  = peak_df['peak_spacing_std'].fillna(pd.Timedelta(0))
#         peak_df['peak_spacing_std']  = peak_df['peak_spacing_std'].fillna(pd.Timedelta(pd.Timedelta.max))
    #-------------------------
    if 'peak_hour_mean' in peak_df.columns:
        peak_df['peak_hour_mean']    = peak_df['peak_hour_mean'].fillna(-1)
    #-------------------------
    return peak_df


def convert_to_seconds_in_peak_df(
    peak_df
):
    r"""
    """
    cols_to_convert = ['peak_width_mean', 'peak_width_std', 'peak_spacing_mean', 'peak_spacing_std']
    for col in cols_to_convert:
        if col in peak_df.columns:
            peak_df[col]   = peak_df[col].dt.total_seconds()/60
    #-------------------------
    return peak_df

In [15]:
def make_it_funky(df_i, n_entries=500, val_col='value', SN_col='serialnumber'):
    if df_i.shape[0] < n_entries:
        return None
    assert(df_i[SN_col].nunique()==1)
    vals_i = df_i[val_col].sort_index().iloc[:n_entries].tolist()
    return_i = pd.Series(vals_i)
    return_i.name = df_i[SN_col].unique()[0]
    return return_i

In [ ]:
def smooth_peaks_OLD(
    df, 
    value_col, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    r"""
    Do not use too small a value for lag!!!!!
        This can cause the std values to become very small, causing much to be identified as peaks.
        It's not a huge deal when signal_abs_threshold!=0, because this usually gives the algorithm enough
          data to work out the rolling mean and std reliably.
    """
    #-----
    y = df[value_col].tolist()
    #-----
    if len(y) <= lag:
        return np.asarray(y)
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    

    # To avoid possibility of data starting with peak, which can mess up methods (especially if
    #   lag is small), start with first value (after lag) less than the mean
    # NOTE: The mean here includes all data, so includes peaks as well, so should be larger than the
    #         smoothed mean
    i_beg = np.argmax((y[lag:] < np.mean(y))) + lag
    
    non_signal_Y = []
    for i in range(i_beg, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
            filteredY[i] = y[i]
        else:
            # Only looking for peaks, so don't use abs(y-avg)
            if y[i] - avgFilter[i-1] > threshold * stdFilter[i-1]:
                #-------------------------
                # First, set the signal equal to the rolling average
                #-----
                # If avgFilter[i-1]==0, try np.mean(avgFilter)
                # If both==0, use raw value, y[i]
                if avgFilter[i-1]>0:
                    signals[i] = avgFilter[i-1]
                else:
                    if np.mean(avgFilter)>0:
                        signals[i] = np.mean(avgFilter)
                    else:
                        print('Cannot use any averages!')
                        signals[i] = y[i]
                #-------------------------
                # Next, set filteredY
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    tmp_idx = np.random.randint(i-lag, i, 1)[0]
                    tmp_mean = filteredY[tmp_idx]
                    tmp_std = stdFilter[tmp_idx]
                    tmp_val  = np.random.normal(tmp_mean, tmp_std, 1)
                    filteredY[i] = influence * y[i] + (1 - influence) * tmp_val
            else:
                signals[i] = y[i]
                filteredY[i] = y[i]
        #-----
        # Note: As opposed to other thresholding functions, since any peaks have been reduced to average
        #       values, all filteredY are also non_signal_Y (so can be appended here, instead of separately
        #       in if/else statements as in other thresholding functions)
        non_signal_Y.append(filteredY[i])
        #-----
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        
    #-------------------------
    # Now, go back and perform calculation for initial lag entries using the non_signal_Y
    #   for average values
    for i in range(0,i_beg):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
        else:
            if y[i] - np.mean(non_signal_Y) > threshold * np.std(non_signal_Y):
                # Instead of using the mean of non_signal_Y, randomly draw a mean from avgFilter
                signals[i] = np.random.choice(a=avgFilter, size=1)
            else:
                signals[i] = y[i]

    return np.asarray(signals)

In [16]:
def smooth_peaks(
    df, 
    value_col, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    r"""
    Do not use too small a value for lag!!!!!
        This can cause the std values to become very small, causing much to be identified as peaks.
        It's not a huge deal when signal_abs_threshold!=0, because this usually gives the algorithm enough
          data to work out the rolling mean and std reliably.
          
    NOTE: While dealing with the first lag entries after i_beg, we are still susceptible to peaks contaminating
            the data (causing higher means and, more importantly, higher standard deviations)
          THEREFORE, if we are within this regime, instead of using mean and standard devation, I will instead
            use the median and interquartile range (which are much more robust against outliers)
    """
    #-----
    y = df[value_col].tolist()
    #-----
    if len(y) <= lag:
        return np.asarray(y)
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    

    # To avoid possibility of data starting with peak, which can mess up methods (especially if
    #   lag is small), start with first value (after lag) less than the mean
    # NOTE: The mean here includes all data, so includes peaks as well, so should be larger than the
    #         smoothed mean
    i_beg = np.argmax((y[lag:] < np.mean(y))) + lag
    avgFilter[i_beg - 1] = np.median(y[i_beg-lag:i_beg])
    stdFilter[i_beg - 1] = scipy.stats.iqr(y[i_beg-lag:i_beg])
    
    non_signal_Y = []
    for i in range(i_beg, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
            filteredY[i] = y[i]
        else:
            # Only looking for peaks, so don't use abs(y-avg)
            if y[i] - avgFilter[i-1] > threshold * stdFilter[i-1]:
                #-------------------------
                # First, set the signal equal to the rolling average
                #-----
                # If avgFilter[i-1]==0, try np.mean(avgFilter)
                # If both==0, use raw value, y[i]
                if avgFilter[i-1]>0:
                    signals[i] = avgFilter[i-1]
                else:
                    if np.mean(avgFilter)>0:
                        signals[i] = np.mean(avgFilter)
                    else:
                        print('Cannot use any averages!')
                        signals[i] = y[i]
                #-------------------------
                # Next, set filteredY
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    tmp_idx = np.random.randint(i-lag, i, 1)[0]
                    tmp_mean = filteredY[tmp_idx]
                    tmp_std = stdFilter[tmp_idx]
                    tmp_val  = np.random.normal(tmp_mean, tmp_std, 1)
                    filteredY[i] = influence * y[i] + (1 - influence) * tmp_val
            else:
                signals[i] = y[i]
                filteredY[i] = y[i]
        #-----
        # Note: As opposed to other thresholding functions, since any peaks have been reduced to average
        #       values, all filteredY are also non_signal_Y (so can be appended here, instead of separately
        #       in if/else statements as in other thresholding functions)
        non_signal_Y.append(filteredY[i])
        #-----
        if i-i_beg > lag:
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
            stdFilter[i] = scipy.stats.iqr(filteredY[(i-lag+1):i+1])            
        
    #-------------------------
    # Now, go back and perform calculation for initial lag entries using the non_signal_Y
    #   for average values
    for i in range(0,i_beg):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
        else:
            if y[i] - np.mean(non_signal_Y) > threshold * np.std(non_signal_Y):
                # Instead of using the mean of non_signal_Y, randomly draw a mean from avgFilter
                signals[i] = np.random.choice(a=avgFilter, size=1)
            else:
                signals[i] = y[i]

    return np.asarray(signals)

In [17]:
def thresholding_algo_df_i(
    df_i, 
    value_col, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold, 
    smooth_rolling_window, 
    SN_col='serialnumber', 
    PN_col='aep_premise_nb', 
    time_col='starttimeperiod_local', 
    signal_col_out='signal', 
    signal_pos_col_out='signal_pos', 
    signal_binary_col_out='signal_binary', 
    signal_pos_binary_col_out='signal_pos_binary',  
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    if df_i.index.name != time_col:
        assert(time_col in df_i.columns)
        df_i = df_i.set_index(time_col, drop=True)
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    # First, smooth out the peaks so the un-biased rolling mean and std can be formed
    smooth = smooth_peaks(
        df=df_i, 
        value_col=value_col, 
        lag=lag, 
        threshold=threshold, 
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold
    )
    smooth_col = f'{value_col}_smooth'
    df_i[smooth_col] = smooth
    #-------------------------
    # Generate smooth rolling mean and std
    smooth_roll_mean_col = f'{value_col}_smooth_roll_mean'
    smooth_roll_std_col  = f'{value_col}_smooth_roll_std'
    df_i[smooth_roll_mean_col] = df_i[smooth_col].rolling(smooth_rolling_window, center=True).mean()
    df_i[smooth_roll_std_col]  = df_i[smooth_col].rolling(smooth_rolling_window, center=True).std()
    #-----
    # Drop the NaN values at front and back of df_i naturally resulting from rolling window methods
    df_i = df_i.dropna(subset=[smooth_roll_mean_col, smooth_roll_std_col]).copy()
    #-------------------------
    # Generate the signal column, which is defined for each point as the number of (rolling) std deviations 
    #   away from the (rolling) mean.
    # Also generate the binary version of the signal, which is simply whether or not the number of std away from
    #   the mean is outside of the threshold.
    # Finally, build the positive versions of the above two, which set any values less than the mean equal to 0 (since
    #   we are looking for positive peaks)
    #-----
    df_i[signal_col_out] = (df_i[value_col] - df_i[smooth_roll_mean_col])/df_i[smooth_roll_std_col]

    # Probably really only want points which are above the rolling mean, since we're looking for peaks
    # For now, keep both signal and signal_pos
    df_i[signal_pos_col_out] = df_i[signal_col_out]
    df_i.loc[df_i[signal_pos_col_out]<0, signal_pos_col_out] = 0

    # Binary results are yes/no of whether peak is found
    df_i[signal_binary_col_out]     = np.abs(df_i[signal_col_out]) > threshold
    df_i[signal_pos_binary_col_out] = np.abs(df_i[signal_pos_col_out]) > threshold
    
    #-------------------------
    # Generate a couple test series for possible use
    df_i['test_final'] = df_i[value_col]
    df_i.loc[df_i[signal_binary_col_out]==0, 'test_final'] = df_i.loc[df_i[signal_binary_col_out]==0, smooth_roll_mean_col]
    #-----
    df_i['test_final2'] = df_i[value_col]
    df_i.loc[df_i[signal_binary_col_out]==0, 'test_final2'] = 0
    #-----
    df_i['test_final_signal'] = df_i[signal_col_out]
    df_i.loc[df_i[signal_binary_col_out]==0, 'test_final_signal'] = 0
    
    #-------------------------
    # To save space, don't keep all of columns in df_i, only really keep those build here (plus SN, PN, value)
    cols_to_keep = [
        SN_col, 
        PN_col, 
        value_col, 
        smooth_col, 
        smooth_roll_mean_col, 
        smooth_roll_std_col, 
        signal_col_out, 
        signal_pos_col_out, 
        signal_binary_col_out, 
        signal_pos_binary_col_out, 
        'test_final', 
        'test_final2', 
        'test_final_signal'
    ]
    return df_i[cols_to_keep]

In [18]:
def build_peak_features_for_df_i(
    df_i, 
    signal_group_col='signal_grp', 
    value_col='value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    if time_col not in df_i.columns:
        assert(time_col==df_i.index.name)
        df_i[time_col] = df_i.index
    assert(time_col in df_i.columns)
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()].copy()
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    df_i_signals = df_i_signals.reset_index(drop=True)
    #-------------------------
    # Features split into:
    #   1. Using all data in peaks pooled together
    #   2. First grouping by signal group, then extracting features
    #-------------------------
    # Features (1):
    peak_mean = df_i_signals[value_col].mean()
    peak_std  = df_i_signals[value_col].std()
    #-------------------------
    # Features (2):
    #-----
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_max_std      = df_i_signals_gpd[(value_col, 'max')].std()
    #-----
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_width_std    = df_i_signals_gpd[(time_col, 'max_m_min')].std()
    #-----
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    peak_spacing_std  = df_i_signals_gpd[(time_col, 'mean')].diff().std()
    #-----
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_mean':         peak_mean, 
        'peak_std':          peak_std, 
        #-----
        'peak_max_mean':     peak_max_mean, 
        'peak_max_std':      peak_max_std, 
        #-----
        'peak_width_mean':   peak_width_mean, 
        'peak_width_std':    peak_width_std, 
        #-----
        'peak_spacing_mean': peak_spacing_mean,
        'peak_spacing_std':  peak_spacing_std,
        #-----
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [19]:
fig_num=0

In [20]:
build_ami_df = False
files_dir = r'C:\Users\s346557\Documents\LocalData\EVs\Data'
pkl_save_dir =  r'C:\Users\s346557\Documents\LocalData\EVs'

In [21]:
if build_ami_df:
    ami_df = GenAn.read_df_from_csv_dir_batches(
        files_dir=files_dir, 
        file_path_glob=r'*.csv'
    )
    #-------------------------
    ami_df = ami_df[
        (ami_df['aep_derived_uom']=='KWH') & 
        (ami_df['aep_srvc_qlty_idntfr']=='TOTAL')
    ].copy()
    #-----
    ami_df['timezoneoffset'] = ami_df['starttimeperiod'].str[-6:]
    #-------------------------
    ami_df = AMINonVee.perform_std_initiation_and_cleaning(
        ami_df, 
        timestamp_col=None
    )
    #-----
    ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df, 
        time_col='starttimeperiod', 
        placement_col='starttimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df, 
        time_col='endtimeperiod', 
        placement_col='endtimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df=ami_df.set_index('starttimeperiod_local', drop=False)
    #-------------------------
    ami_df.to_pickle(os.path.join(pkl_save_dir, 'ami_df.pkl'))
else:
    ami_df = pd.read_pickle(os.path.join(pkl_save_dir, 'ami_df.pkl'))
#-------------------------
all_trff_dfs = pd.read_pickle(os.path.join(pkl_save_dir, 'all_trff_dfs.pkl'))

In [22]:
ami_df['aep_premise_nb'].nunique()

1712

1712

In [23]:
all_trff_dfs['PREM_NB'].nunique()

1957

1957

In [24]:
all_trff_dfs_found     = all_trff_dfs[all_trff_dfs['PREM_NB'].isin(ami_df['aep_premise_nb'].unique().tolist())]
all_trff_dfs_not_found = all_trff_dfs[~all_trff_dfs['PREM_NB'].isin(ami_df['aep_premise_nb'].unique().tolist())]

In [25]:
print(all_trff_dfs_found['PREM_NB'].nunique())
print(all_trff_dfs_not_found['PREM_NB'].nunique())

1712
245
1712
245


In [26]:
print(ami_df['aep_premise_nb'].nunique())
print(ami_df['serialnumber'].nunique())

1712
1712
2021
2021


In [27]:
ami_df = ami_df.groupby('aep_premise_nb', as_index=False, group_keys=False).apply(
    lambda x: remove_ev_submeter_in_pair(x)
)

In [28]:
print(ami_df['aep_premise_nb'].nunique())
print(ami_df['serialnumber'].nunique())

1673
1673
1673
1673


In [29]:
ami_df_resamples = AMINonVee.build_time_resampled_dfs(
    ami_df, 
    base_freq='15T', 
    freqs=['H', '2H', '3H', '4H'], 
    other_grouper_cols=['serialnumber', 'aep_premise_nb']
)

In [30]:
ami_df[['starttimeperiod_local', 'starttimeperiod_utc']]

,starttimeperiod_local,starttimeperiod_utc
starttimeperiod_local,,
2022-05-23 03:45:00,2022-05-23 03:45:00,2022-05-23 07:45:00
2022-05-23 20:30:00,2022-05-23 20:30:00,2022-05-24 00:30:00
2022-05-23 13:00:00,2022-05-23 13:00:00,2022-05-23 17:00:00
2022-05-23 07:00:00,2022-05-23 07:00:00,2022-05-23 11:00:00
2022-05-23 21:15:00,2022-05-23 21:15:00,2022-05-24 01:15:00
...,...,...
2023-01-22 17:00:00,2023-01-22 17:00:00,2023-01-22 23:00:00
2023-01-27 01:30:00,2023-01-27 01:30:00,2023-01-27 07:30:00
2023-01-22 22:30:00,2023-01-22 22:30:00,2023-01-23 04:30:00


,starttimeperiod_local,starttimeperiod_utc
starttimeperiod_local,,
2022-05-23 03:45:00,2022-05-23 03:45:00,2022-05-23 07:45:00
2022-05-23 20:30:00,2022-05-23 20:30:00,2022-05-24 00:30:00
2022-05-23 13:00:00,2022-05-23 13:00:00,2022-05-23 17:00:00
2022-05-23 07:00:00,2022-05-23 07:00:00,2022-05-23 11:00:00
2022-05-23 21:15:00,2022-05-23 21:15:00,2022-05-24 01:15:00
...,...,...
2023-01-22 17:00:00,2023-01-22 17:00:00,2023-01-22 23:00:00
2023-01-27 01:30:00,2023-01-27 01:30:00,2023-01-27 07:30:00
2023-01-22 22:30:00,2023-01-22 22:30:00,2023-01-23 04:30:00


In [31]:
evs_prems = all_trff_dfs[all_trff_dfs['EV']==1]['PREM_NB'].unique().tolist()
non_prems = all_trff_dfs[all_trff_dfs['EV']==0]['PREM_NB'].unique().tolist()

# aep_premise_nb in ami_df is of type object (i.e., a string), whereas PREM_NB in trff_df is int64
evs_prems = [str(x) for x in evs_prems]
non_prems = [str(x) for x in non_prems]
#-----
ami_df_evs = ami_df[ami_df['aep_premise_nb'].isin(evs_prems)].copy()
ami_df_non = ami_df[ami_df['aep_premise_nb'].isin(non_prems)].copy()
#----
assert(ami_df.shape[0]==ami_df_evs.shape[0]+ami_df_non.shape[0])

In [32]:
print(f'ami_df_evs.shape[0] = {ami_df_evs.shape[0]}')
print(f'ami_df_non.shape[0] = {ami_df_non.shape[0]}')

ami_df_evs.shape[0] = 2808398
ami_df_non.shape[0] = 6591357
ami_df_evs.shape[0] = 2808398
ami_df_non.shape[0] = 6591357


In [33]:
print(f"ami_df_evs['aep_premise_nb'].nunique() = {ami_df_evs['aep_premise_nb'].nunique()}")
print(f"ami_df_non['aep_premise_nb'].nunique() = {ami_df_non['aep_premise_nb'].nunique()}")

ami_df_evs['aep_premise_nb'].nunique() = 492
ami_df_evs['aep_premise_nb'].nunique() = 492
ami_df_non['aep_premise_nb'].nunique() = 1181
ami_df_non['aep_premise_nb'].nunique() = 1181


In [34]:
ami_df_resamples_evs = {}
ami_df_resamples_non = {}
for freq_i, dfs_i in ami_df_resamples.items():
    df_i = dfs_i['df']
    ami_df_evs_i = df_i[df_i['aep_premise_nb'].isin(evs_prems)].copy()
    ami_df_non_i = df_i[df_i['aep_premise_nb'].isin(non_prems)].copy()   
    #-----
    assert(df_i.shape[0]==ami_df_evs_i.shape[0]+ami_df_non_i.shape[0])
    assert(freq_i not in ami_df_resamples_evs.keys())
    assert(freq_i not in ami_df_resamples_non.keys())
    ami_df_resamples_evs[freq_i] = ami_df_evs_i
    ami_df_resamples_non[freq_i] = ami_df_non_i

# ===========================================================
# ===========================================================

# GRID SEARCH

In [35]:
def make_model(input_shape):
    num_classes=2
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=32, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=16, kernel_size=25, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=8, kernel_size=10, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)
    
    conv4 = keras.layers.Conv1D(filters=4, kernel_size=10, padding="same")(conv3)
    conv4 = keras.layers.BatchNormalization()(conv4)
    conv4 = keras.layers.ReLU()(conv4)

    gap = keras.layers.GlobalAveragePooling1D()(conv4)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

In [36]:
grid_results_df_empty = pd.DataFrame(columns=[
    'lag', 
    'threshold', 
    'signal_abs_threshold', 
    'smooth_rolling_window', 
    #-----
    'acc_train_keras', 
    'prec_train_keras', 
    'rec_train_keras', 
    'f1_train_keras', 
    
    'acc_test_keras', 
    'prec_test_keras', 
    'rec_test_keras', 
    'f1_test_keras', 
    #-----
    'acc_train_forest', 
    'prec_train_forest', 
    'rec_train_forest', 
    'f1_train_forest', 
    
    'acc_test_forest', 
    'prec_test_forest', 
    'rec_test_forest', 
    'f1_test_forest', 
    #-----
    'acc_train_svc', 
    'prec_train_svc', 
    'rec_train_svc', 
    'f1_train_svc', 
    
    'acc_test_svc', 
    'prec_test_svc', 
    'rec_test_svc', 
    'f1_test_svc', 
    #-----
    'acc_train_logreg', 
    'prec_train_logreg', 
    'rec_train_logreg', 
    'f1_train_logreg', 
    
    'acc_test_logreg', 
    'prec_test_logreg', 
    'rec_test_logreg', 
    'f1_test_logreg'    
])

In [37]:
ami_df_evs_fit_OG = ami_df_resamples_evs['H'].copy()
ami_df_non_fit_OG = ami_df_resamples_non['H'].copy()
#-----
value_col = 'mean_TRS value'
funky_n_entries=200

fit_signal_col_keras='test_final'
# fit_signal_col_keras='test_final2'

fit_signal_col_peaks = 'test_final'
# fit_signal_col_peaks = 'signal'

In [38]:
lags = [24, 36, 48]
thresholds = [2, 3, 5, 10]
signal_abs_thresholds = [0, 1, 2]
smooth_rolling_windows = [24, 36, 48]
influence=0

# lags = [24, 48]
# thresholds = [3]
# signal_abs_thresholds = [1]
# smooth_rolling_windows = [24]

In [39]:
grid_lists = [
    lags, 
    thresholds, 
    signal_abs_thresholds, 
    smooth_rolling_windows
]

In [40]:
start=time.time()
grid_results_df = grid_results_df_empty.copy()
for i, grid_i in enumerate(list(itertools.product(*grid_lists))):
    start_i = time.time()
    lag                   = grid_i[0]
    threshold             = grid_i[1]
    signal_abs_threshold  = grid_i[2]
    smooth_rolling_window = grid_i[3]
    #-------------------------
    print(f"{i} of {len(list(itertools.product(*grid_lists)))}")
    print(f'lag                   = {lag}')
    print(f'threshold             = {threshold}')
    print(f'signal_abs_threshold  = {signal_abs_threshold}')
    print(f'smooth_rolling_window = {smooth_rolling_window}')
    #-------------------------
    res_i = dict()
    res_i['lag']                   = lag
    res_i['threshold']             = threshold
    res_i['signal_abs_threshold']  = signal_abs_threshold
    res_i['smooth_rolling_window'] = smooth_rolling_window
    #-------------------------
    ami_df_evs_fit = ami_df_evs_fit_OG.groupby(['serialnumber'], as_index=False, group_keys=False).apply(
        lambda x: thresholding_algo_df_i(
            df_i=x, 
            value_col=value_col, 
            lag=lag, 
            threshold=threshold, 
            influence=influence, 
            signal_abs_threshold=signal_abs_threshold, 
            smooth_rolling_window=smooth_rolling_window
        )
    )
    ami_df_non_fit = ami_df_non_fit_OG.groupby(['serialnumber'], as_index=False, group_keys=False).apply(
        lambda x: thresholding_algo_df_i(
            df_i=x, 
            value_col=value_col, 
            lag=lag, 
            threshold=threshold, 
            influence=influence, 
            signal_abs_threshold=signal_abs_threshold, 
            smooth_rolling_window=smooth_rolling_window
        )
    )

    #----------------------------------------------------------------------------------------------------
    # NEW METHOD KERAS
    #----------------------------------------------------------------------------------------------------
    keras_df_evs = ami_df_evs_fit.reset_index(drop=True).groupby(['serialnumber']).apply(
        lambda x: make_it_funky(
            df_i=x, 
            n_entries=funky_n_entries, 
            val_col=fit_signal_col_keras, 
            SN_col='serialnumber'
        )
    )
    keras_df_non = ami_df_non_fit.reset_index(drop=True).groupby(['serialnumber']).apply(
        lambda x: make_it_funky(
            df_i=x, 
            n_entries=funky_n_entries, 
            val_col=fit_signal_col_keras, 
            SN_col='serialnumber'
        )
    )
    #-------------------------
    keras_df_evs['target']=1
    keras_df_non['target']=0
    keras_df = pd.concat([keras_df_evs, keras_df_non])
    keras_df=keras_df.dropna()
    #-------------------------
    keras_df_train, keras_df_test = train_test_split(keras_df, test_size=0.33, random_state=42)
    #-------------------------
    X_train = keras_df_train[[x for x in keras_df_train.columns.tolist() if x != 'target']].values
    y_train = keras_df_train['target'].values
    X_train=X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    #-----
    X_test = keras_df_test[[x for x in keras_df_test.columns.tolist() if x != 'target']].values
    y_test = keras_df_test['target'].values
    X_test=X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    #-------------------------
    model = make_model(input_shape=X_train.shape[1:])
    #-------------------------
    epochs = 500
    batch_size = 32

    callbacks = [
        keras.callbacks.ModelCheckpoint(
            "best_model.h5", save_best_only=True, monitor="val_loss"
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
        ),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
    ]
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_split=0.2,
        verbose=0,
    )
    #-------------------------
    model = keras.models.load_model("best_model.h5")
    #-------------------------
    y_pred_train = model.predict(X_train)
    y_pred_train = np.argmax(y_pred_train, axis=1)
    #-----
    res_i['acc_train_keras']  = accuracy_score(y_train, y_pred_train)
    res_i['prec_train_keras'] = precision_score(y_train, y_pred_train)
    res_i['rec_train_keras']  = recall_score(y_train, y_pred_train)
    res_i['f1_train_keras']   = f1_score(y_train, y_pred_train)
    #-------------------------
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    #-----
    res_i['acc_test_keras']  = accuracy_score(y_test, y_pred)
    res_i['prec_test_keras'] = precision_score(y_test, y_pred)
    res_i['rec_test_keras']  = recall_score(y_test, y_pred)
    res_i['f1_test_keras']   = f1_score(y_test, y_pred)

    #----------------------------------------------------------------------------------------------------
    # NEW METHOD PEAKS
    #----------------------------------------------------------------------------------------------------
    peak_df_evs = ami_df_evs_fit.groupby(['serialnumber'], as_index=False, group_keys=False).apply(
        lambda x: set_signal_groups_in_df_i(
            df_i=x, 
            SN_col='serialnumber',
            signal_col='signal_binary', 
            return_signal_group_col='signal_grp'        
        )
    )
    #-----
    peak_df_non = ami_df_non_fit.groupby(['serialnumber'], as_index=False, group_keys=False).apply(
        lambda x: set_signal_groups_in_df_i(
            df_i=x, 
            SN_col='serialnumber',
            signal_col='signal_binary', 
            return_signal_group_col='signal_grp'        
        )
    )
    #-------------------------
    peak_df_evs = peak_df_evs.groupby(['serialnumber'], as_index=False, group_keys=False).apply(
        lambda x: build_peak_features_for_df_i(
            df_i=x, 
            signal_group_col='signal_grp', 
            value_col=fit_signal_col_peaks,
            SN_col='serialnumber', 
            time_col='starttimeperiod_local',        
        )
    )
    #-----
    peak_df_non = peak_df_non.groupby(['serialnumber'], as_index=False, group_keys=False).apply(
        lambda x: build_peak_features_for_df_i(
            df_i=x, 
            signal_group_col='signal_grp', 
            value_col=fit_signal_col_peaks, 
            SN_col='serialnumber', 
            time_col='starttimeperiod_local',        
        )
    )
    #-------------------------
    peak_df_evs['target']=1
    peak_df_non['target']=0
    #-------------------------
    peak_df=pd.concat([peak_df_evs, peak_df_non])
    peak_df=peak_df.sample(frac=1)
    #-------------------------
    # peak_df_OG = peak_df.copy()
    #-------------------------
    # peak_df = peak_df_OG.copy()
    #-------------------------
    peak_df = fill_na_in_peak_df(peak_df)
    peak_df = convert_to_seconds_in_peak_df(peak_df)
    peak_df = peak_df.drop(columns=['serialnumber'])
    #-------------------------
    # peak_df = peak_df.drop(columns=[
    #     'peak_mean', 'peak_std', 
    #     'peak_max_std', 
    #     'peak_width_std', 
    #     'peak_spacing_std'
    # ])
    #-------------------------
    # Remove inf values
    print(f"Before inf removal, peak_df.shape[0] = {peak_df.shape[0]}")
    peak_df = peak_df.loc[np.isinf(peak_df[[x for x in peak_df.columns if x!='serialnumber']]).sum(axis=1)==0].copy()
    print(f"After inf removal, peak_df.shape[0] = {peak_df.shape[0]}")
    #-------------------------
    peak_df_train, peak_df_test = train_test_split(peak_df, test_size=0.33, random_state=42)
    #-------------------------
    X_train = peak_df_train[[x for x in peak_df_train.columns.tolist() if x!='target']]
    y_train = peak_df_train['target']
    #-----
    X_test = peak_df_test[[x for x in peak_df_test.columns.tolist() if x!='target']]
    y_test = peak_df_test['target']
    #--------------------------------------------------
    # Random Forest
    #--------------------------------------------------
    forest_clf = RandomForestClassifier(n_estimators = 10, max_depth=None, n_jobs=None)
    forest_clf.fit(X_train, y_train)
    #-------------------------
    y_pred_train = forest_clf.predict(X_train)
    res_i['acc_train_forest']  = accuracy_score(y_train, y_pred_train)
    res_i['prec_train_forest'] = precision_score(y_train, y_pred_train)
    res_i['rec_train_forest']  = recall_score(y_train, y_pred_train)
    res_i['f1_train_forest']   = f1_score(y_train, y_pred_train)
    #-------------------------
    y_pred = forest_clf.predict(X_test)
    res_i['acc_test_forest']  = accuracy_score(y_test, y_pred)
    res_i['prec_test_forest'] = precision_score(y_test, y_pred)
    res_i['rec_test_forest']  = recall_score(y_test, y_pred)
    res_i['f1_test_forest']   = f1_score(y_test, y_pred)

    #--------------------------------------------------
    # SVC Forest
    #--------------------------------------------------
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)
    #-------------------------
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    #-------------------------
    y_pred_train = clf.predict(X_train)
    res_i['acc_train_svc']  = accuracy_score(y_train, y_pred_train)
    res_i['prec_train_svc'] = precision_score(y_train, y_pred_train)
    res_i['rec_train_svc']  = recall_score(y_train, y_pred_train)
    res_i['f1_train_svc']   = f1_score(y_train, y_pred_train)
    #-------------------------
    y_pred = clf.predict(X_test)
    res_i['acc_test_svc']  = accuracy_score(y_test, y_pred)
    res_i['prec_test_svc'] = precision_score(y_test, y_pred)
    res_i['rec_test_svc']  = recall_score(y_test, y_pred)
    res_i['f1_test_svc']   = f1_score(y_test, y_pred)

    #--------------------------------------------------
    # Logistic Regression
    #--------------------------------------------------
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    #-------------------------
    y_pred_train = logreg.predict(X_train)
    res_i['acc_train_logreg']  = accuracy_score(y_train, y_pred_train)
    res_i['prec_train_logreg'] = precision_score(y_train, y_pred_train)
    res_i['rec_train_logreg']  = recall_score(y_train, y_pred_train)
    res_i['f1_train_logreg']   = f1_score(y_train, y_pred_train)
    #-------------------------
    y_pred = logreg.predict(X_test)
    res_i['acc_test_logreg']  = accuracy_score(y_test, y_pred)
    res_i['prec_test_logreg'] = precision_score(y_test, y_pred)
    res_i['rec_test_logreg']  = recall_score(y_test, y_pred)
    res_i['f1_test_logreg']   = f1_score(y_test, y_pred)

    #----------------------------------------------------------------------------------------------------
    #----------------------------------------------------------------------------------------------------
    grid_results_df = pd.concat([grid_results_df, pd.DataFrame(res_i, index=[grid_results_df.shape[0]])])
    print(time.time()-start_i)
    print()
    
print(time.time()-start)

0 of 108
lag                   = 24
threshold             = 2
signal_abs_threshold  = 0
smooth_rolling_window = 24
0 of 108
lag                   = 24
threshold             = 2
signal_abs_threshold  = 0
smooth_rolling_window = 24
Epoch 102: early stopping
Epoch 102: early stopping
18/18 [==============================] - 0s 6ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
351.10175704956055

1 of 108
lag                   = 24
threshold             = 2
signal_abs_threshold  = 0
smooth_rolling_window = 36
351.10175704956055

1 of 108
lag                   = 24
threshold             = 2
signal_abs_threshold  = 0
smooth_rolling_window = 36
Epoch 93: early stopping
Epoch 93: early stopping
18/18 [==============================] - 0s 6ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.sha

18/18 [==============================] - 0s 8ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
275.5525243282318

12 of 108
lag                   = 24
threshold             = 3
signal_abs_threshold  = 1
smooth_rolling_window = 24
275.5525243282318

12 of 108
lag                   = 24
threshold             = 3
signal_abs_threshold  = 1
smooth_rolling_window = 24
Epoch 121: early stopping
Epoch 121: early stopping
18/18 [==============================] - 0s 8ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
296.2243399620056

13 of 108
lag                   = 24
threshold             = 3
signal_abs_threshold  = 1
smooth_rolling_window = 36
296.2243399620056

13 of 108
lag                   = 24
threshold             = 3
signal_

260.4662663936615

23 of 108
lag                   = 24
threshold             = 5
signal_abs_threshold  = 1
smooth_rolling_window = 48
Epoch 88: early stopping
Epoch 88: early stopping
18/18 [==============================] - 0s 8ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
258.0058205127716

24 of 108
lag                   = 24
threshold             = 5
signal_abs_threshold  = 2
smooth_rolling_window = 24
258.0058205127716

24 of 108
lag                   = 24
threshold             = 5
signal_abs_threshold  = 2
smooth_rolling_window = 24
Epoch 206: early stopping
Epoch 206: early stopping
18/18 [==============================] - 0s 7ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
341.63764929771423

25 of 108
lag                   = 24
threshold             = 5
signal_abs_threshold  = 2
smooth_rolling_wi

Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
469.8236403465271

35 of 108
lag                   = 24
threshold             = 10
signal_abs_threshold  = 2
smooth_rolling_window = 48
469.8236403465271

35 of 108
lag                   = 24
threshold             = 10
signal_abs_threshold  = 2
smooth_rolling_window = 48
Epoch 105: early stopping
Epoch 105: early stopping
18/18 [==============================] - 0s 7ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
282.00968623161316

36 of 108
lag                   = 36
threshold             = 2
signal_abs_threshold  = 0
smooth_rolling_window = 24
282.00968623161316

36 of 108
lag                   = 36
threshold             = 2
signal_abs_threshold  = 0
smooth_rolling_window = 24
Epo

Epoch 90: early stopping
Epoch 90: early stopping
18/18 [==============================] - 0s 6ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
220.2338743209839

47 of 108
lag                   = 36
threshold             = 3
signal_abs_threshold  = 0
smooth_rolling_window = 48
220.2338743209839

47 of 108
lag                   = 36
threshold             = 3
signal_abs_threshold  = 0
smooth_rolling_window = 48
Epoch 107: early stopping
Epoch 107: early stopping
18/18 [==============================] - 0s 8ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
248.5122983455658

48 of 108
lag                   = 36
threshold             = 3
signal_abs_threshold  = 1
smooth_rolling_window = 24
248.5122983455658

48 of 108
lag       

...

18/18 [==============================] - 0s 8ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
453.97717022895813

70 of 108
lag                   = 36
threshold             = 10
signal_abs_threshold  = 2
smooth_rolling_window = 36
453.97717022895813

70 of 108
lag                   = 36
threshold             = 10
signal_abs_threshold  = 2
smooth_rolling_window = 36
Epoch 147: early stopping
Epoch 147: early stopping
18/18 [==============================] - 0s 6ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
285.6342701911926

71 of 108
lag                   = 36
threshold             = 10
signal_abs_threshold  = 2
smooth_rolling_window = 48
285.6342701911926

71 of 108
lag                   = 36
threshold             = 10
s

266.6828751564026

81 of 108
lag                   = 48
threshold             = 3
signal_abs_threshold  = 0
smooth_rolling_window = 24
Epoch 102: early stopping
Epoch 102: early stopping
18/18 [==============================] - 0s 9ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
369.87444496154785

82 of 108
lag                   = 48
threshold             = 3
signal_abs_threshold  = 0
smooth_rolling_window = 36
369.87444496154785

82 of 108
lag                   = 48
threshold             = 3
signal_abs_threshold  = 0
smooth_rolling_window = 36
Epoch 89: early stopping
Epoch 89: early stopping
18/18 [==============================] - 0s 7ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
258.5117554664612

83 of 108
lag     

Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
391.0564384460449

93 of 108
lag                   = 48
threshold             = 5
signal_abs_threshold  = 1
smooth_rolling_window = 24
391.0564384460449

93 of 108
lag                   = 48
threshold             = 5
signal_abs_threshold  = 1
smooth_rolling_window = 24
Epoch 186: early stopping
Epoch 186: early stopping
18/18 [==============================] - 0s 6ms/step
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
Before inf removal, peak_df.shape[0] = 1672
After inf removal, peak_df.shape[0] = 1672
360.96975016593933

94 of 108
lag                   = 48
threshold             = 5
signal_abs_threshold  = 1
smooth_rolling_window = 36
360.96975016593933

94 of 108
lag                   = 48
threshold             = 5
signal_abs_threshold  = 1
smooth_rolling_window = 36
Epoch

OSError: Unable to create file (file signature not found)

OSError: Unable to create file (file signature not found)

In [41]:
grid_results_df

,lag,threshold,signal_abs_threshold,smooth_rolling_window,acc_train_keras,prec_train_keras,rec_train_keras,f1_train_keras,acc_test_keras,prec_test_keras,...,rec_test_svc,f1_test_svc,acc_train_logreg,prec_train_logreg,rec_train_logreg,f1_train_logreg,acc_test_logreg,prec_test_logreg,rec_test_logreg,f1_test_logreg
0,24,2,0,24,0.881508,0.925764,0.648318,0.762590,0.859745,0.931373,...,0.645570,0.728571,0.857143,0.837209,0.646707,0.729730,0.865942,0.813433,0.689873,0.746575
1,24,2,0,36,0.874887,0.956098,0.601227,0.738230,0.852190,0.897196,...,0.632184,0.707395,0.866071,0.841463,0.650943,0.734043,0.831522,0.804511,0.614943,0.697068
2,24,2,0,48,0.912691,0.952569,0.739264,0.832470,0.859489,0.861789,...,0.680851,0.757396,0.853571,0.818182,0.592105,0.687023,0.853261,0.868966,0.670213,0.756757
3,24,2,1,24,0.907540,0.995575,0.688073,0.813743,0.843352,0.883495,...,0.590361,0.690141,0.878571,0.868217,0.687117,0.767123,0.847826,0.820312,0.632530,0.714286
4,24,2,1,36,0.886589,0.927350,0.665644,0.775000,0.852190,0.851240,...,0.634286,0.720779,0.871429,0.847390,0.665615,0.745583,0.846014,0.830882,0.645714,0.726688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,48,5,0,36,0.889289,0.963470,0.647239,0.774312,0.841241,0.862385,...,0.640523,0.736842,0.849107,0.824427,0.637168,0.718802,0.844203,0.786325,0.601307,0.681481
92,48,5,0,48,0.850585,0.939560,0.524540,0.673228,0.821168,0.935897,...,0.628205,0.717949,0.858929,0.842308,0.651786,0.734899,0.842391,0.800000,0.589744,0.678967
93,48,5,1,24,0.880610,0.953271,0.623853,0.754159,0.834244,0.911111,...,0.617143,0.734694,0.872321,0.848000,0.668770,0.747795,0.846014,0.862903,0.611429,0.715719
94,48,5,1,36,0.883888,0.949772,0.638037,0.763303,0.850365,0.881818,...,0.664706,0.760943,0.873214,0.846154,0.683230,0.756014,0.864130,0.857143,0.670588,0.752475


,lag,threshold,signal_abs_threshold,smooth_rolling_window,acc_train_keras,prec_train_keras,rec_train_keras,f1_train_keras,acc_test_keras,prec_test_keras,...,rec_test_svc,f1_test_svc,acc_train_logreg,prec_train_logreg,rec_train_logreg,f1_train_logreg,acc_test_logreg,prec_test_logreg,rec_test_logreg,f1_test_logreg
0,24,2,0,24,0.881508,0.925764,0.648318,0.762590,0.859745,0.931373,...,0.645570,0.728571,0.857143,0.837209,0.646707,0.729730,0.865942,0.813433,0.689873,0.746575
1,24,2,0,36,0.874887,0.956098,0.601227,0.738230,0.852190,0.897196,...,0.632184,0.707395,0.866071,0.841463,0.650943,0.734043,0.831522,0.804511,0.614943,0.697068
2,24,2,0,48,0.912691,0.952569,0.739264,0.832470,0.859489,0.861789,...,0.680851,0.757396,0.853571,0.818182,0.592105,0.687023,0.853261,0.868966,0.670213,0.756757
3,24,2,1,24,0.907540,0.995575,0.688073,0.813743,0.843352,0.883495,...,0.590361,0.690141,0.878571,0.868217,0.687117,0.767123,0.847826,0.820312,0.632530,0.714286
4,24,2,1,36,0.886589,0.927350,0.665644,0.775000,0.852190,0.851240,...,0.634286,0.720779,0.871429,0.847390,0.665615,0.745583,0.846014,0.830882,0.645714,0.726688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,48,5,0,36,0.889289,0.963470,0.647239,0.774312,0.841241,0.862385,...,0.640523,0.736842,0.849107,0.824427,0.637168,0.718802,0.844203,0.786325,0.601307,0.681481
92,48,5,0,48,0.850585,0.939560,0.524540,0.673228,0.821168,0.935897,...,0.628205,0.717949,0.858929,0.842308,0.651786,0.734899,0.842391,0.800000,0.589744,0.678967
93,48,5,1,24,0.880610,0.953271,0.623853,0.754159,0.834244,0.911111,...,0.617143,0.734694,0.872321,0.848000,0.668770,0.747795,0.846014,0.862903,0.611429,0.715719
94,48,5,1,36,0.883888,0.949772,0.638037,0.763303,0.850365,0.881818,...,0.664706,0.760943,0.873214,0.846154,0.683230,0.756014,0.864130,0.857143,0.670588,0.752475


In [42]:
grid_results_df.sort_values(by=['prec_test_keras'], ascending=False)

,lag,threshold,signal_abs_threshold,smooth_rolling_window,acc_train_keras,prec_train_keras,rec_train_keras,f1_train_keras,acc_test_keras,prec_test_keras,...,rec_test_svc,f1_test_svc,acc_train_logreg,prec_train_logreg,rec_train_logreg,f1_train_logreg,acc_test_logreg,prec_test_logreg,rec_test_logreg,f1_test_logreg
39,36,2,1,24,0.860862,0.921569,0.574924,0.708098,0.846995,0.945055,...,0.691358,0.764505,0.868750,0.858824,0.663636,0.748718,0.867754,0.839695,0.679012,0.750853
74,48,2,0,48,0.856886,0.956284,0.536810,0.687623,0.844891,0.945055,...,0.650000,0.740506,0.853571,0.818966,0.608974,0.698529,0.836957,0.851562,0.605556,0.707792
79,48,2,2,36,0.884788,0.958333,0.634969,0.763838,0.857664,0.940000,...,0.642384,0.721190,0.857143,0.829091,0.668622,0.740260,0.858696,0.841121,0.596026,0.697674
92,48,5,0,48,0.850585,0.939560,0.524540,0.673228,0.821168,0.935897,...,0.628205,0.717949,0.858929,0.842308,0.651786,0.734899,0.842391,0.800000,0.589744,0.678967
54,36,5,0,24,0.844704,0.932584,0.507645,0.657426,0.821494,0.935065,...,0.726190,0.774603,0.847321,0.822785,0.601852,0.695187,0.842391,0.795620,0.648810,0.714754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,36,10,0,36,0.875788,0.919643,0.631902,0.749091,0.837591,0.834783,...,0.400000,0.550459,0.822321,0.890710,0.476608,0.620952,0.811594,0.838235,0.380000,0.522936
31,24,10,1,36,0.887489,0.931330,0.665644,0.776386,0.841241,0.826446,...,0.479042,0.608365,0.856250,0.918367,0.553846,0.690979,0.811594,0.838710,0.467066,0.600000
70,36,10,2,36,0.877588,0.895833,0.659509,0.759717,0.839416,0.825000,...,0.360947,0.523605,0.798214,0.929204,0.325077,0.481651,0.797101,0.952381,0.355030,0.517241
34,24,10,2,36,0.851485,0.930481,0.533742,0.678363,0.806569,0.806122,...,0.387097,0.533333,0.818750,0.918750,0.436202,0.591549,0.807971,0.855072,0.380645,0.526786


,lag,threshold,signal_abs_threshold,smooth_rolling_window,acc_train_keras,prec_train_keras,rec_train_keras,f1_train_keras,acc_test_keras,prec_test_keras,...,rec_test_svc,f1_test_svc,acc_train_logreg,prec_train_logreg,rec_train_logreg,f1_train_logreg,acc_test_logreg,prec_test_logreg,rec_test_logreg,f1_test_logreg
39,36,2,1,24,0.860862,0.921569,0.574924,0.708098,0.846995,0.945055,...,0.691358,0.764505,0.868750,0.858824,0.663636,0.748718,0.867754,0.839695,0.679012,0.750853
74,48,2,0,48,0.856886,0.956284,0.536810,0.687623,0.844891,0.945055,...,0.650000,0.740506,0.853571,0.818966,0.608974,0.698529,0.836957,0.851562,0.605556,0.707792
79,48,2,2,36,0.884788,0.958333,0.634969,0.763838,0.857664,0.940000,...,0.642384,0.721190,0.857143,0.829091,0.668622,0.740260,0.858696,0.841121,0.596026,0.697674
92,48,5,0,48,0.850585,0.939560,0.524540,0.673228,0.821168,0.935897,...,0.628205,0.717949,0.858929,0.842308,0.651786,0.734899,0.842391,0.800000,0.589744,0.678967
54,36,5,0,24,0.844704,0.932584,0.507645,0.657426,0.821494,0.935065,...,0.726190,0.774603,0.847321,0.822785,0.601852,0.695187,0.842391,0.795620,0.648810,0.714754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,36,10,0,36,0.875788,0.919643,0.631902,0.749091,0.837591,0.834783,...,0.400000,0.550459,0.822321,0.890710,0.476608,0.620952,0.811594,0.838235,0.380000,0.522936
31,24,10,1,36,0.887489,0.931330,0.665644,0.776386,0.841241,0.826446,...,0.479042,0.608365,0.856250,0.918367,0.553846,0.690979,0.811594,0.838710,0.467066,0.600000
70,36,10,2,36,0.877588,0.895833,0.659509,0.759717,0.839416,0.825000,...,0.360947,0.523605,0.798214,0.929204,0.325077,0.481651,0.797101,0.952381,0.355030,0.517241
34,24,10,2,36,0.851485,0.930481,0.533742,0.678363,0.806569,0.806122,...,0.387097,0.533333,0.818750,0.918750,0.436202,0.591549,0.807971,0.855072,0.380645,0.526786
